### Preparação de dados

No contexto de machine learning, o pré-processamento de dados é uma etapa fundamental que antecede a construção e o treinamento de modelos. Raramente os dados brutos coletados de fontes reais estão prontos para uso imediato: eles costumam conter valores ausentes, ruídos, inconsistências, escalas distintas e formatos inadequados para os algoritmos de aprendizado.

O objetivo do pré-processamento é transformar esses dados brutos em um conjunto mais limpo, consistente e informativo, capaz de representar adequadamente o problema a ser resolvido. Entre as principais tarefas dessa etapa estão a limpeza dos dados, o tratamento de valores faltantes, a normalização ou padronização de atributos, a codificação de variáveis categóricas e a seleção ou extração de características relevantes.

Alguns exemplos de técnicas de pré-processamento amplamente utilizadas incluem:

- Tratamento de valores ausentes, como remoção de registros incompletos ou imputação usando média, mediana, moda ou modelos preditivos.
- Normalização e padronização, por meio de técnicas como Min-Max Scaling e Standardization (z-score), que ajustam as escalas das variáveis numéricas.
- Codificação de variáveis categóricas, utilizando métodos como One-Hot Encoding ou Label Encoding.
- Detecção e tratamento de outliers, com técnicas estatísticas (como IQR e desvio padrão) ou métodos baseados em modelos.
- Seleção de atributos, empregando filtros estatísticos, wrapper methods ou técnicas embutidas (embedded methods), a fim de reduzir dimensionalidade e ruído.
- Extração de características, como Principal Component Analysis (PCA), que transforma os dados em um novo espaço de menor dimensão.

Um bom pré-processamento impacta diretamente o desempenho dos modelos de machine learning, pois algoritmos aprendem padrões com base na qualidade das informações fornecidas. Dados mal preparados podem levar a modelos imprecisos, enviesados ou instáveis, enquanto um pré-processamento adequado contribui para maior eficiência, melhor generalização e resultados mais confiáveis.

#### 1. Import de bibliotecas

In [42]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize, StandardScaler, OneHotEncoder, LabelEncoder

#### 2. Criação de um df de exemplo

In [8]:
df = pd.DataFrame({"nome": ['Peter', 'Bruce', "T'Challa"],
                    "simbolo": ['aranha', ',morcego', 'pantera'],
                    "idade": [22, pd.NaT, 25]                    
                    })

df

,nome,simbolo,idade
0,Peter,aranha,22
1,Bruce,",morcego",NaT
2,T'Challa,pantera,25


#### 3. Valores ausentes

Opções para tratar:
* remoção: eliminar linhas ou colunas com valores ausentes, caso a quantidade seja significativa
* imputação: preencher os valores ausentes com valores estimados, como média, mediana ou valores previstos por um modelo auxiliar

##### 3.1 Remoção

In [5]:
df.dropna() #não altera o df, só visualiza

,nome,simbolo,idade
0,Peter,aranha,22
2,T'Challa,pantera,25


In [ ]:
df.dropna(inplace=True) #altera o df

##### 3.2 Imputação 

In [9]:
#Dados com valores ausentes 
dados = np.array([[1,2,3,np.nan],
                [4,np.nan,6,7],
                [8,2,5,7],
                [8,11,10,10]
                ])

dados

array([[ 1.,  2.,  3., nan],
       [ 4., nan,  6.,  7.],
       [ 8.,  2.,  5.,  7.],
       [ 8., 11., 10., 10.]])

In [15]:
#Cria imputador com estratégia de substituir pela média - por coluna/feature
imputador1 = SimpleImputer(strategy='mean')

dados_imputados1 = imputador1.fit_transform(dados)

#Note que os valores vazios foram substituidos pela média
dados_imputados1

array([[ 1.,  2.,  3.,  8.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  2.,  5.,  7.],
       [ 8., 11., 10., 10.]])

In [16]:
#Cria imputador com estratégia de substituir pelo valor mais frequente - por coluna/feature
imputador2 = SimpleImputer(strategy='most_frequent')

dados_imputados2 = imputador2.fit_transform(dados)

#Note que os valores vazios foram substituidos pela média
dados_imputados2

array([[ 1.,  2.,  3.,  7.],
       [ 4.,  2.,  6.,  7.],
       [ 8.,  2.,  5.,  7.],
       [ 8., 11., 10., 10.]])

#### 4. Identificação de Outliers

A distância interquartílica (IQR – Interquartile Range) é uma métrica estatística amplamente utilizada para detecção de outliers em conjuntos de dados. Ela se baseia na distribuição dos quartis, que dividem os dados em quatro partes iguais.

Como funciona:

1. Calcular os quartis:
𝑄1 = 1º quartil (25% dos dados estão abaixo deste valor)
Q3 = 3º quartil (75% dos dados estão abaixo deste valor)

2. Calcular a distância interquartílica (IQR):
IQR=𝑄3−𝑄1

3. Definir limites para detecção de outliers:
Os outliers são valores que ficam fora do intervalo:
Limite Inferior=𝑄1−1.5×IQR
Limite Superior = 𝑄3+1.5×IQR 

Qualquer ponto fora desses limites é considerado um outlier.

Vantagens do IQR:
- Baseado em quartis, menos sensível a valores extremos do que média e desvio padrão.
- Simples de calcular e interpretar.
- Funciona bem para distribuições assimétricas.

Limitações:
- Assumindo o fator 1.5, pode não detectar outliers em distribuições muito assimétricas ou com caudas pesadas.
- Não fornece informação sobre a gravidade do outlier, apenas indica que ele é extremo em relação à distribuição central.

Contexto estatístico:

O fator 1.5 usado na regra da distância interquartílica (IQR) tem um fundamento estatístico relacionado à distribuição normal. Em uma distribuição aproximadamente normal, cerca de 50% dos dados estão dentro do intervalo definido pelos quartis (Q1 a Q3). Multiplicar o IQR por 1.5 estende esse intervalo para cobrir a maior parte dos dados típicos, mas ainda marca como outliers os valores muito afastados da mediana, que são raros nessa distribuição.

Na prática, isso significa que, para dados que seguem uma distribuição normal, o fator 1.5 identifica como outliers valores que estão aproximadamente além de 2 desvios padrão do centro dos 50% centrais. Esse valor foi escolhido porque representa um bom equilíbrio: ele é suficientemente sensível para detectar pontos extremos, mas não tão rígido a ponto de classificar como outliers variações naturais do conjunto de dados.

Essa regra se tornou um padrão porque funciona bem em muitas situações, mesmo quando os dados não são perfeitamente normais, e é a base do boxplot, onde os “bigodes” estendem-se até 1.5×IQR e os pontos fora dele são considerados outliers.

In [18]:
dados = np.array([1,2,3,4,5,100])
dados

array([  1,   2,   3,   4,   5, 100])

In [22]:
q1 = np.percentile(dados, 25)
q3 = np.percentile(dados, 75)
iqr = q3 - q1 

print(f'Com q1 = {q1} e q3 = {q3}, IQR = {iqr}')

limite_inf = q1 - 1.5 * iqr
limite_sup = q3 + 1.5 * iqr

print(f'Os limites inferiores e superiores foram, respectivamente, de {limite_inf} e {limite_sup}')


Com q1 = 2.25 e q3 = 4.75, IQR = 2.5
Os limites inferiores e superiores foram, respectivamente, de -1.5 e 8.5


In [26]:
outliers = np.where((dados < limite_inf) | (dados > limite_sup))[0]
outliers 

array([5], dtype=int64)

In [30]:
#posicao do elemento em dados
outliers[0]

5

In [31]:
#Valor do elemento em dados
dados[outliers[0]]

100

In [32]:
#Remoção do outlier
np.delete(dados, 5)

array([1, 2, 3, 4, 5])

#### 5. Normalização dos dados

A normalização de dados é uma etapa fundamental, especialmente quando os atributos possuem escalas diferentes. O objetivo da normalização é transformar os dados para uma mesma escala, sem distorcer suas distribuições, permitindo que algoritmos que dependem de distância ou gradiente — como KNN, SVM, redes neurais e regressão logística — funcionem corretamente.

Por que normalizar os dados?
- Muitos algoritmos assumem que todas as variáveis têm igual importância. Se uma variável varia de 0 a 1.000 e outra de 0 a 1, a primeira dominará o aprendizado se os dados não forem normalizados.
- Facilita a convergência de algoritmos de otimização, como o gradiente descendente.
- Ajuda na interpretação e comparação de diferentes atributos.

Principais técnicas de normalização:

**Min-Max Scaling**: 
- Transforma os valores para um intervalo fixo, normalmente entre 0 e 1, usando o valor mínimo e máximo de cada atributo.
- Valores extremos são ajustados ao limite do intervalo.
- Muito usada quando os dados têm limites conhecidos.


**Z-score (Padronização ou Standardization)**
- Centraliza os dados em torno da média e ajusta pela variância, de forma que os atributos tenham média zero e desvio padrão 1.
- Ideal para dados com distribuições aproximadamente normais.
- Reduz impacto de escalas diferentes sem necessariamente limitar os valores a um intervalo fixo.

    
**Robust Scaling**
- Usa mediana e IQR em vez de média e desvio padrão, tornando a transformação menos sensível a outliers.

!! Observações importantes:
- Nem todos os algoritmos precisam de normalização: árvores de decisão e random forests, por exemplo, não são afetadas por escalas diferentes.
- A escolha da técnica depende do tipo de dados e do algoritmo que será usado.

In [34]:
dados = np.array([2,3,5,6,7,8,7,6,0])
dados

array([2, 3, 5, 6, 7, 8, 7, 6, 0])

In [35]:
dados_normalizados = normalize([dados], norm = 'max')
dados_normalizados

array([[0.25 , 0.375, 0.625, 0.75 , 0.875, 1.   , 0.875, 0.75 , 0.   ]])

1. Normalização L1 --> Também chamada de norma Manhattan ou norma da soma absoluta. Cada valor do vetor é dividido pela soma dos valores absolutos de todos os elementos do vetor.
2. Normalização L2 --> Também chamada de norma Euclidiana. Cada valor do vetor é dividido pela raiz quadrada da soma dos quadrados de todos os elementos do vetor.
3. Normalização max --> Cada valor do vetor é dividido pelo maior valor absoluto do vetor. Resultado: o maior valor do vetor normalizado é 1 (ou -1, se negativo).

| Tipo    | O que faz                                   | Resultado         |
| ------- | ------------------------------------------- | ----------------- |
| **L1**  | Divide pelo somatório dos valores absolutos | Soma absoluta = 1 |
| **L2**  | Divide pela raiz da soma dos quadrados      | Magnitude = 1     |
| **Max** | Divide pelo valor máximo do vetor           | Maior valor = 1   |


#### 6. Padronização dos dados

A padronização de dados é uma técnica de pré-processamento usada para transformar os atributos de um conjunto de dados de forma que eles tenham média zero e desvio padrão igual a um. Cada valor do atributo é transformado subtraindo-se a média e dividindo-se pelo desvio padrão do atributo.

Por que padronizar os dados?
- Muitos algoritmos de aprendizado de máquina, como regressão linear, SVM, K-means e redes neurais, funcionam melhor quando os atributos têm mesma escala e distribuição centralizada.
- Evita que variáveis com valores grandes dominem o aprendizado em relação a variáveis com valores menores.
- Facilita a convergência de algoritmos que usam gradiente descendente, tornando o treinamento mais estável.

In [39]:
dados = np.array([[1,2,3],[4,5,6],[7,8,9]])
dados

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [40]:
padronizador = StandardScaler()

In [41]:
dados_padronizados = padronizador.fit_transform(dados)
dados_padronizados

array([[-1.22474487, -1.22474487, -1.22474487],
       [ 0.        ,  0.        ,  0.        ],
       [ 1.22474487,  1.22474487,  1.22474487]])

7. Codificação de variáveis categóricas

A codificação de variáveis categóricas é uma técnica de pré-processamento de dados usada em aprendizado de máquina para transformar variáveis não numéricas (categóricas) em valores numéricos, que os algoritmos conseguem interpretar.

Por que codificar?
- A maioria dos algoritmos de machine learning não consegue trabalhar diretamente com textos ou categorias (como “vermelho”, “azul”, “verde”).
- A codificação permite que essas informações sejam representadas numericamente, sem perder o significado das categorias.

##### 7.1 One-Hot Encoding

In [43]:
dados_categoricos = np.array(['A', 'B', 'C', 'A', 'B'])
dados_categoricos

array(['A', 'B', 'C', 'A', 'B'], dtype='<U1')

In [44]:
codific_one_hot = OneHotEncoder()

In [48]:
dados_one_hot = codific_one_hot.fit_transform(dados_categoricos.reshape(-1,1))
print(dados_one_hot.toarray())

#Note que A, B e C viraram colunas e 1 representa que aquela variavel se aplica e 0 que não se aplica

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


##### 7.2 Label Encoding

In [50]:
codific_label = LabelEncoder()

In [52]:
dados_label = codific_label.fit_transform(dados_categoricos.reshape(-1,1))
print(dados_label)

[0 1 2 0 1]
